# Generar tablas

Correr esta notebook completa para generar las tablas que luego vamos a procesar en el análisis de regresión lineal en R

In [1]:
#
# Este boilerplate es para que Python encuentre el código 
#
%matplotlib inline

from nbhelpers import add_source_path

add_source_path("..")


In [2]:
%matplotlib inline

import pylab
import config
import logging
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from session import SessionMapper, load_session
from IPython.display import display
from pandasql import sqldf
from social import get_social_variables

social_variables = get_social_variables()

pysqldf = lambda q: sqldf(q, globals())

logger = logging.getLogger('main')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)


mapper = SessionMapper()

sessions = [load_session(i) for i in range(1,13)]

display(social_variables)

Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_01/s01.objects.1.tasks not found. Building from scratch
INFO:main:Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_01/s01.objects.1.tasks not found. Building from scratch
Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_02/s02.objects.1.tasks not found. Building from scratch
INFO:main:Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_02/s02.objects.1.tasks not found. Building from scratch
Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_03/s03.objects.1.tasks not found. Building from scratch
INFO:main:Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_03/s03.objects.1.tasks not found. Building from scratch
Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_04/s04.objects.1.tasks not found. Building from scratch
INFO:main:Pickle for session /home/jmperez/projects/tesis/da

,session,task,speaker,bored_with_game,difficult_for_partner_to_speak,contributes_to_successful_completion,engaged_in_game,gives_encouragement,making_self_clear,planning_what_to_say,dislikes_partner
hitid,,,,,,,,,,,
g10t1,10,1,0,0,0,5,5,3,4,4,0
g10t10,10,10,0,0,2,4,4,2,5,3,4
g10t11,10,11,0,1,0,5,5,5,5,5,0
g10t12,10,12,0,0,0,5,5,4,5,3,0
g10t13,10,13,0,0,0,4,4,5,4,3,1
g10t14,10,14,0,2,0,5,4,5,4,3,0
g10t2,10,2,0,0,3,5,5,2,4,3,0
g10t3,10,3,0,0,0,4,4,5,4,4,3
g10t4,10,4,0,1,3,3,4,2,1,3,1


Generamos las tablas para cada feature:

In [4]:
import os
from analyzers import Analyzer


features = ["F0_MEAN", 
            "F0_MAX",
            "ENG_MAX", 
            "ENG_MEAN", 
            "SYLLABES_AVG", 
            'PHONEMES_AVG',
            'NOISE_TO_HARMONICS_RATIO',
            'SOUND_ALL_LOCAL_JITTER',
            'SOUND_ALL_LOCAL_SHIMMER',
            'SOUND_VOICED_LOCAL_JITTER',
            'SOUND_VOICED_LOCAL_SHIMMER'
]
all_features_analysis = pd.DataFrame(columns=features, dtype=float)

social_vars = [var for var in social_variables.columns if var not in ["session", "task", "speaker"]]

for feature in features:
    analysis = Analyzer(feature).analyze(sessions)
    joined_analysis = pysqldf("""
    SELECT * from analysis INNER JOIN social_variables ON 
        analysis.session=social_variables.session AND 
        analysis.task=social_variables.task AND
        analysis.speaker=social_variables.speaker""")
    
    table_path = os.path.join(config.TABLES_DIR, "%s.csv" % feature)
    joined_analysis.to_csv(table_path)
        

There was an error calling /home/jmperez/projects/tesis/scripts/voice-analysis.praat with interval [528.4725, 528.945]
ERROR:main:There was an error calling /home/jmperez/projects/tesis/scripts/voice-analysis.praat with interval [528.4725, 528.945]
?-() has no defined phonemes. Defaulting to 0
?-() has no defined phonemes. Defaulting to 0
-?() has no defined phonemes. Defaulting to 0
?-() has no defined phonemes. Defaulting to 0
?-() has no defined phonemes. Defaulting to 0
?-() has no defined phonemes. Defaulting to 0
uu-(uu) has no defined phonemes. Defaulting to 0
uu(uu) has no defined phonemes. Defaulting to 0
?-() has no defined phonemes. Defaulting to 0
?-() has no defined phonemes. Defaulting to 0
uu(uu) has no defined phonemes. Defaulting to 0
?-() has no defined phonemes. Defaulting to 0
uu(uu) has no defined phonemes. Defaulting to 0
uu(uu) has no defined phonemes. Defaulting to 0
uu(uu) has no defined phonemes. Defaulting to 0
uu(uu) has no defined phonemes. Defaulting to 0


In [5]:
for session in sessions:
    mapper.save(session)